In [1]:
import os
from bs4 import BeautifulSoup as BS
import time
import datetime
import requests
from multiprocessing import Pool, Manager, cpu_count
from manager import *
CPU = cpu_count() * 8

years = ['2014',] #'2015']#, '2014', '2015', '2016', '2017']
months = ['01', ]#'02', '03', '04', '05', '06']#, '07', '08', '09', '10', '11', '12']
days = ['01','02']#, '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
    

In [2]:
def download_file(args):
    url, out_dir, my_dict, y, d, m = args
    r = requests.get(url[0])
    html_doc = r.text
    filename = os.path.join(out_dir, url[0][url[0].rfind("/") + 1:] + ".txt")
    #filename = os.path.join(out_dir, url[0]+".txt")
    soup = BS(html_doc, 'html.parser')
    tag = soup.select_one(".StandardArticleBody_body")
    #text = '\n'.join([e.get_text() for e in soup.find_all('p')])
    title = soup.html.title.string.strip().split(" | ")[0]
    not_found = soup.select_one("#sectionTitle")
    if tag != None:
        date = str(datetime.datetime.strptime("{} {} {} {}".format(m, d, y, url[1]), '%m %d %Y %I:%M%p'))
        my_dict[url[0]] = [date, title, tag.text, "Reuters"]
        #add_news(date, title, tag.text, url[0], "Reuters")
    elif not_found:
        print(url, not_found.text)
    else:
        print(url, "Error")
    return my_dict

def retrieve_links(date):
    url = "https://www.reuters.com/resources/archive/us/{}.html".format(date)
    r = requests.get(url)
    html_doc = r.text
    soup = BS(html_doc, 'html.parser')
    span_list = soup.select(".headlineMed")
    links_time = [[str(el).split('>')[1].split("=")[1].replace('"', ''), str(el).split()[-2]] for el in span_list]
    urls2parse = []
    for link in links_time:
        if not link[0].startswith("http://www.reuters.com/news/video/"):
            urls2parse.append(link)
    return urls2parse

if __name__ == "__main__":
    start = time.time()
    for y in years:
        for m in months:
            for d in days:
                try:
                    datetime.datetime(year=int(y),month=int(m),day=int(d))
                except ValueError:
                    print("No such date: {}/{}/{}".format(y, m, d))
                    continue
                cpu_count = CPU
                date = y+m+d
                start = time.time()
                urls2parse = retrieve_links(date)
                #print(urls2parse)
                out_dir = "//home/ostapkharysh/Documents/bt_data/" + date
                #if not os.path.exists(out_dir):
                #    os.makedirs(out_dir)
                manager = Manager()
                my_dict = manager.dict()
                args = [(url, out_dir, my_dict, y, d, m) for url in urls2parse]
                with Pool(cpu_count) as p:
                    p.map(download_file, args)
                for key in my_dict.keys():
                    add_news(my_dict[key][0], my_dict[key][1], my_dict[key][2], key, my_dict[key][3])
                # d = my_dict._getvalue()
                print(time.time() - start)
                print(len(d), len(urls2parse))
    print((time.time() - start) /60)

0.8186721801757812
2 0
0.6186215877532959
2 0
0.01031820774078369
